In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import gmaps.datasets
# import requests
# import numpy as np

# Import the API key
from config import g_key

In [2]:
# Store the CSV into a DataFrame
vacation_data_df = pd.read_csv("weather_database/WeatherPy_vacation.csv")
# vacation_data_df

In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [4]:
# Review data types to ensure appropriateness
vacation_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [5]:
# Ensure completeness -> not concerned about missing hotels, will just avoid picking those two cities
vacation_data_df.count()

City_ID                26
City                   26
Country                26
Max Temp               26
Current Description    26
Lat                    26
Lng                    26
Hotel Name             24
dtype: int64

In [6]:
# Review vacation_data_df for geographically close cities (min. 4)
vacation_data_df.sort_values(by=['Country'])

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,272,Launceston,AU,50.00,light rain,-41.45,147.17,Hotel Grand Chancellor Launceston
11,224,Pastavy,BY,63.75,light rain,55.12,26.83,"Общежитие УО ""Поставский государственный колледж"""
12,228,Ancud,CL,50.00,light rain,-41.87,-73.82,HOTEL ANCUD DON LUCAS
18,379,Tulu Bolo,ET,63.59,light rain,8.67,38.22,Kiya Sime Hotel
24,549,Bako,ET,67.19,light rain,5.78,36.57,Arkisha Sefer
8,134,Bedele,ET,60.04,light rain,8.46,36.35,Central Hotel
25,561,Gimbi,ET,61.72,moderate rain,9.17,35.83,Sam home
9,164,Dharchula,IN,63.52,light rain,29.85,80.53,Mahara Hotel
20,435,Lashio,MM,66.09,light rain,22.93,97.75,Hotel CF Lashio - Burmese Only
16,274,Vardo,NO,48.20,light rain,70.37,31.11,Vardø Hotel AS


In [7]:
# Create four separate DataFrames for four geographically close cities (all within Ethiopia)
destination_1_df = vacation_data_df.loc[[8]].copy().reset_index(drop=True)
destination_2_df = vacation_data_df.loc[[18]].copy().reset_index(drop=True)
destination_3_df = vacation_data_df.loc[[24]].copy().reset_index(drop=True)
destination_4_df = vacation_data_df.loc[[25]].copy().reset_index(drop=True)

In [8]:
# Get the lat-long pairs from each DataFrame
bedele = (destination_1_df["Lat"].values[0],destination_1_df["Lng"].values[0])
tulu_bolo = (destination_2_df["Lat"].values[0],destination_2_df["Lng"].values[0])
bako = (destination_3_df["Lat"].values[0],destination_3_df["Lng"].values[0])
gimbi = (destination_4_df["Lat"].values[0],destination_4_df["Lng"].values[0])

In [9]:
# Create directions map
fig = gmaps.figure()
ethiopia_tour = gmaps.directions_layer(bako,tulu_bolo, waypoints=[bedele,gimbi], travel_mode='DRIVING')
fig.add_layer(ethiopia_tour)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Create a new DataFrame to house all Ethiopian locations
ethiopia_hotels = [destination_1_df,destination_2_df,destination_3_df,destination_4_df]
ethiopia_hotel_df = pd.concat(ethiopia_hotels).reset_index(drop=True)
ethiopia_hotel_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,134,Bedele,ET,60.04,light rain,8.46,36.35,Central Hotel
1,379,Tulu Bolo,ET,63.59,light rain,8.67,38.22,Kiya Sime Hotel
2,549,Bako,ET,67.19,light rain,5.78,36.57,Arkisha Sefer
3,561,Gimbi,ET,61.72,moderate rain,9.17,35.83,Sam home


In [11]:
# Create the pop up box framework
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and high of {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
ethiopia_hotel_info = [info_box_template.format(**row) for index, row in ethiopia_hotel_df.iterrows()]

In [12]:
# Create directions map with pop-up markers
locations = ethiopia_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure()
ethiopia_tour = gmaps.directions_layer(bako,tulu_bolo, waypoints=[bedele,gimbi], travel_mode='DRIVING', show_markers=False)
marker_layer = gmaps.marker_layer(locations, info_box_content=ethiopia_hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(ethiopia_tour)
fig

Figure(layout=FigureLayout(height='420px'))